In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('cleaned.csv')

# Display the first ten rows of the data frame to examine if it is an individual-level data set
print(df.head(10))

FileNotFoundError: ignored

In [ ]:
rvar_list =['quality']
df_sample1 = df.drop(columns=rvar_list)

nvar_list = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 
             'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 
  'sulphates', 'alcohol']
cvar_list = ['good_quality_1']

In [ ]:
df_sample3 = df_sample1.copy()
df_sample3[cvar_list] = df_sample1[cvar_list].astype('category')
df_sample3[nvar_list] = df_sample1[nvar_list].astype('float64')

In [1]:
# Required package: scikit-learn. Package name in Python: sklearn
# Required subpackage: model_selection. Required function name: train_test_split
from sklearn.model_selection import train_test_split

# Placeholder variables: df4partition, testpart_size
# test_size specifies the percentage for the test partition
df4partition = df_sample3
testpart_size = 0.2

# random_state specifies the seed for random number generator. 
# random_state = 1 unless otherwised noted
df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

print(df_nontestData)


NameError: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'good_quality_1'
y = df_nontestData[DV]
X = df_nontestData.drop(columns=[DV])

# Run Logistic regression with pre-specified penalty level (alpha)
# Placeholder variable: alpha
alpha = 10

# The Logistic regression results are put into a model object clf
clf = LogisticRegression(C=1/alpha, penalty='l1', solver='saga', max_iter=200, random_state=1).fit(X,y)

# A user-defined function summary_coef
# to display the estimated coefficients of a model candidate obtained by the Logistic Regression analysis
def summary_coef(model_object):
  n_predictors = X.shape[1]
  model_coef = pd.DataFrame(model_object.coef_.reshape(1, n_predictors), columns=X.columns.values)
  model_coef['Intercept'] = model_object.intercept_
  return model_coef.transpose()

print(summary_coef(clf))



                             0
fixed_acidity         0.142207
volatile_acidity     -0.352573
citric_acid          -0.060331
residual_sugar        0.666360
chlorides            -0.350936
free_sulfur_dioxide   0.142947
total_sulfur_dioxide -0.018424
density              -0.676578
pH                    0.248412
sulphates             0.141409
alcohol               0.712785
Intercept            -1.647365


## AUC

In [ ]:
# Run Logistic regression with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of the penalty levels we will search for the optimal penalty level, 
# i.e., the level that leads to the best model candidate
# We search the optimal alpha within [min_alpha, max_alpha]
min_alpha = 0.01
max_alpha = 100

# C is the inverse of alpha. LogisticRegression and LogisticRegressionCV require inverse alpha 
# as the parameter values for specifying penalty levels
max_C = 1/min_alpha
min_C = 1/max_alpha

# We further discretize the continuous C range [min_C, max_C] into n individual points of C
# We train n model candidates each of which corresponds to one individual C point
n_candidates = 5000

import numpy as np

# We store the list of individual C points into C_list
C_list = list(np.linspace(min_C, max_C, num=n_candidates))

clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, scoring='roc_auc', penalty='l1', solver='saga', max_iter=200, random_state=1, n_jobs=-1).fit(X,y)

# Display the estimated coefficients of the final selected model
print(summary_coef(clf_optimal))

# Display the optimal alpha that yields the final selected model (the best model candidate)
print(1/clf_optimal.C_)



                             0
fixed_acidity         0.469865
volatile_acidity     -0.390498
citric_acid          -0.095696
residual_sugar        1.525274
chlorides            -0.339358
free_sulfur_dioxide   0.150111
total_sulfur_dioxide  0.007699
density              -1.979349
pH                    0.501027
sulphates             0.231539
alcohol               0.160923
Intercept            -1.717265
[0.0736846]


In [ ]:
y_test_actual = df_testData[DV]

X_test = df_testData.drop(columns=[DV])

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test_actual, clf_optimal.predict_proba(X_test)[:,1]))

0.8007738212341793


## Accuracy

In [ ]:
clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, scoring='accuracy', penalty='l1', solver='saga', max_iter=200, random_state=1, n_jobs=-1).fit(X,y)

# Display the estimated coefficients of the final selected model
print(summary_coef(clf_optimal))

# Display the optimal alpha that yields the final selected model (the best model candidate)
print(1/clf_optimal.C_)



                             0
fixed_acidity         0.212820
volatile_acidity     -0.360576
citric_acid          -0.068137
residual_sugar        0.850918
chlorides            -0.348254
free_sulfur_dioxide   0.144718
total_sulfur_dioxide -0.013618
density              -0.955375
pH                    0.303004
sulphates             0.160990
alcohol               0.593869
Intercept            -1.660374
[7.69159756]


In [ ]:
# Display the confusion matrix over the test partition
from sklearn import metrics
metrics.confusion_matrix(y_test_actual, clf_optimal.predict(X_test))

array([[718,  41],
       [166,  55]], dtype=int64)

In [ ]:
clf_optimal.score(X_test, y_test_actual )

0.7887755102040817